# Part 1

In [243]:
import json
import cv2 as cv
import numpy as np

# part 1:

def load_obj_each_frame(data_file):
    with open(data_file, 'r') as file:
        frame_dict = json.load(file)
    return frame_dict

def alpha_beta_filter(initial_position, initial_velocity, alpha, beta, observations, dt=1):
    estimated_position = initial_position
    estimated_velocity = initial_velocity

    estimates = []

    for observation in observations:
        # Prediction step
        predicted_position = estimated_position + estimated_velocity * dt
        predicted_velocity = estimated_velocity

        # Update step (if observation is available)
        if observation != [-1, -1]:
            residual = observation - predicted_position
            estimated_position = predicted_position + alpha * residual
            estimated_velocity = predicted_velocity + (beta * residual) / dt
        else:
            # If observation is missing, use the prediction
            estimated_position = predicted_position
            estimated_velocity = predicted_velocity

        #print(estimated_position)

        estimates.append([estimated_position, estimated_velocity])

    return estimates

def draw_target_object_center(video_file, obj_centers):
    count = 0
    cap = cv.VideoCapture(video_file)
    ok, image = cap.read()

    # Initialize Alpha-Beta filter parameters
    initial_position = np.array([313, 229])  # Initial guess for position
    initial_velocity = np.array([-0.47328952, -0.3911483 ])  # Initial guess for velocity
    
    alpha = 0.4 #0.85  # Position update factor, Alpha bigger, 
    beta = 0.0005 #0.005  # Velocity update factor
    
    filtered_estimates = alpha_beta_filter(initial_position, initial_velocity, alpha, beta, obj_centers)

    # Save in JSON file
    aa=[x[0] for x in filtered_estimates]
    part_1_object_tracking = [[int(round(x)), int(round(y))] for x, y in aa]
    output_data = {"obj": part_1_object_tracking}
    with open('part_1_object_tracking.json', 'w', encoding='utf-8') as file:
        json.dump(output_data, file, ensure_ascii=False, indent=None)
    
    print("Position were saved as part_1_object_tracking.json")

    vidwrite = cv.VideoWriter("part_1_demo.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700,500))


    while ok:
        if count < len(filtered_estimates):
            pos, _ = filtered_estimates[count]
            pos_x, pos_y = pos
        else:
            break

        count += 1

        # Resize the image to match coordinates scaling
        image = cv.resize(image, (700, 500))
        
        # Draw the circle at the estimated position
        #if pos_x != -1 and pos_y != -1:
        #    image = cv.circle(image, (int(pos_x), int(pos_y)), 1, (0, 0, 255), 2)

        # Draw the Line
        for i in range(count):
            pos, _ = filtered_estimates[i]
            pos_x, pos_y = pos
            image = cv.circle(image, (int(pos_x), int(pos_y)), 1, (0, 0, 255), 2)
        
        vidwrite.write(image)
        ok, image = cap.read()

    

    vidwrite.release()



In [244]:
frame_dict = load_obj_each_frame("object_to_track.json")
video_file = "commonwealth.mp4"
draw_target_object_center(video_file,frame_dict['obj'])

Position were saved as part_1_object_tracking.json


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


# Part 2

In [ ]:
# part 2:

def draw_object(object_dict,image,color = (0, 255, 0), thickness = 2,c_color= \
                (255, 0, 0)):
    # draw box
    x = object_dict['x_min']
    y = object_dict['y_min']
    width = object_dict['width']
    height = object_dict['height']
    image = cv.rectangle(image, (x, y), (x + width, y + height), color, thickness)
    return image

def draw_objects_in_video(video_file,frame_dict):
    count = 0
    cap = cv.VideoCapture(video_file)
    frames = []
    ok, image = cap.read()
    vidwrite = cv.VideoWriter("part_2_demo.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700,500))
    while ok:
        ######!!!!#######
        image = cv.resize(image, (700, 500)) # make sure your video is resize to this size, otherwise the coords in the data file won't work !!!
        ######!!!!#######
        obj_list = frame_dict[str(count)]
        for obj in obj_list:
          image = draw_object(obj,image)
        vidwrite.write(image)
        count+=1
        ok, image = cap.read()
    vidwrite.release()



In [ ]:
frame_dict = load_obj_each_frame("frame_dict.json")
video_file = "commonwealth.mp4"
draw_objects_in_video(video_file,frame_dict)